In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
url='https://www.worldcubeassociation.org/results/records?region=Czech%20Republic'
wca='https://www.worldcubeassociation.org'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

In [4]:
results = soup.find('div', id='results-list')
table = results.find('table', class_='wca-results')
print(table.prettify())

<table class="table table-condensed wca-results table-hover table-striped table-greedy-last-column" data-locale="en">
 <thead>
  <tr>
   <th class="type">
    Type
   </th>
   <th class="name">
    Name
   </th>
   <th class="result">
    Result
   </th>
   <th class="country">
    Representing
   </th>
   <th class="competition">
    Competition
   </th>
   <th class="solves" colspan="5">
    Solves
   </th>
   <!-- Extra column for .table-greedy-last-column -->
   <th>
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td class="type">
    Single
   </td>
   <td class="name">
    <a href="/persons/2015GROH02">
     Matěj Grohmann
    </a>
   </td>
   <td class="result">
    4.97
   </td>
   <td class="country">
    <span class="fi fi-cz">
    </span>
    Czech Republic
   </td>
   <td class="competition">
    <span class="fi fi-pl">
    </span>
    <a href="/competitions/ProjectBedzin2019">
     Project Będzin 2019
    </a>
   </td>
   <td class="blank-cell" colspan="5">
    <br/>
   </t

In [73]:
def get_date(comp_link):
    comp_data = requests.get(wca+comp_link).text
    comp_soup = BeautifulSoup(comp_data, 'html.parser')
    head = comp_soup.find('div', class_='col-md-6')
    date = head.dd.text.strip()

    if '-' in date:
        splitted = date.split()
        if len(splitted) == 5:
            date = f'{splitted[0]} {splitted[3]} {splitted[4]}'
        else:
            date = f'{splitted[3]} {splitted[4]} {splitted[5]}'
        return date
    return date


In [6]:
def get_days(date):
    input_format = '%b %d, %Y'
    date_object = datetime.strptime(date, input_format)
    return (datetime.now() - date_object).days



In [76]:
def get_row(m, event):
    grid = {}
    grid['Event'] = event
    grid['Type'] = m.find('td', class_='type').text.strip()
    grid['Name'] = m.find('td', class_='name').text.strip()
    grid['Result'] = m.find('td', class_='result').text.strip()
    grid['Competition'] = m.find('td', class_='competition').text.strip()
    comp_link = m.find('td', class_='competition').a['href']
    days = get_days(get_date(comp_link))
    grid['Days'] = days
    return grid

In [79]:
data = pd.DataFrame(columns=['Event', 'Type', 'Name', 'Result', 'Competition', 'Days'])

for table in results.find_all('table', class_='wca-results'):
    event = table.parent.previous_sibling.previous_sibling.text.strip()
    single = table.tbody.find('tr')
    g_single = get_row(single, event)
    data = pd.concat([data, pd.DataFrame([g_single])], ignore_index=True)

    avg = single.next_sibling.next_sibling
    if avg is None:
        continue 
    g_avg = get_row(avg, event)
    data = pd.concat([data, pd.DataFrame([g_avg])], ignore_index=True)


In [85]:
data.sort_values(by='Days', ascending=False, ignore_index=True)

,Event,Type,Name,Result,Competition,Days
0,3x3x3 Multi-Blind,Single,Matěj Mužátko,16/18 55:59,Munich Open 2016,2509
1,3x3x3 One-Handed,Average,Matěj Grohmann,11.99,Brno Open 2017,2292
2,3x3x3 Blindfolded,Single,Matěj Mužátko,32.48,Slovakian Open 2017,2215
3,3x3x3 Cube,Average,Matěj Grohmann,7.22,Austrian Open 2019,1683
4,5x5x5 Cube,Single,Matěj Grohmann,47.63,Wrocław Open 2019,1669
5,3x3x3 Cube,Single,Matěj Grohmann,4.97,Project Będzin 2019,1523
6,4x4x4 Cube,Single,Matěj Grohmann,25.20,Project Będzin 2019,1523
7,5x5x5 Cube,Average,Matěj Grohmann,52.42,Kostelec Summer 2019,1515
8,6x6x6 Cube,Single,Matěj Grohmann,1:39.99,Kostelec Summer 2019,1515
9,3x3x3 Fewest Moves,Single,Matěj Grohmann,21,Wrocław Side Challenge 2019,1480
